# Search AiiDA Database Molecule optimizations

In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)
    
from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.calculation.work import WorkCalculation
from aiida.orm.calculation.job import JobCalculation
from aiida.orm import load_node

import ase.io
from base64 import b64encode
import StringIO
import numpy as np
import ipywidgets as ipw
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

from tempfile import NamedTemporaryFile

In [ ]:
############################   START OF PREPROCESSING   ###############################

In [ ]:
PREPROCESS_VERSION = 0.20

def preprocess_newbies():
    qb = QueryBuilder()
    qb.append(WorkCalculation, filters={
        'attributes._process_label': 'MoleculeOptWorkChain',
        'or':[
               {'extras': {'!has_key': 'preprocess_version'}},
               {'extras.preprocess_version': {'<': PREPROCESS_VERSION}},
           ],
    })
    
    for m in qb.all(): # iterall() would interfere with set_extra()
        n = m[0]
        if not n.is_sealed:
            print("Skipping underway workchain PK %d"%n.pk)
            continue
        if 'obsolete' not in n.get_extras():
            n.set_extra('obsolete', False)
        try:
            preprocess_one(n)
            n.set_extra('preprocess_successful', True)
            n.set_extra('preprocess_version', PREPROCESS_VERSION)
            print("Preprocessed PK %d"%n.pk)
        except Exception as e:
            n.set_extra('preprocess_successful', False)
            n.set_extra('preprocess_error', str(e))
            n.set_extra('preprocess_version', PREPROCESS_VERSION)
            print("Failed to preprocess PK %d: %s"%(n.pk, e))

In [ ]:
def preprocess_one(workcalc):
   
    def get_calc_by_label(workcalc, label):
        qb = QueryBuilder()
        qb.append(WorkCalculation, filters={'uuid':workcalc.uuid})
        qb.append(JobCalculation, output_of=WorkCalculation, filters={'label':label})
        #qb.order_by({'calc':[{'id':{'order':'desc'}}]})
        if qb.count() == 0:
            raise(Exception("Could not find %s calculation."%label))
        calc = qb.all()[0][0]
        return calc

    # formula
    structure = workcalc.inp.structure
    ase_struct = structure.get_ase()
    bulk_formula = ase_struct.get_chemical_formula()
    workcalc.set_extra('formula', '{}'.format(bulk_formula))
    workcalc.set_extra('structure_description', structure.description)
    
    # optimized structure
    moleculeopt_calc = get_calc_by_label(workcalc, "molecule_opt") # TODO deal with restarts, check final state
    opt_structure = moleculeopt_calc.out.output_structure
    workcalc.set_extra('opt_structure_uuid', moleculeopt_calc.out.output_structure.uuid)
    workcalc.set_extra('energy', moleculeopt_calc.res.energy)
#    workcalc.set_extra('cell', opt_structure.get_ase().get_cell_lengths_and_angles())
#    workcalc.set_extra('volume', opt_structure.get_ase().get_volume())

    # thumbnail
    thumbnail = render_thumbnail(opt_structure.get_ase())
    workcalc.set_extra('thumbnail', thumbnail)
    
    

In [ ]:
def render_thumbnail(atoms):
    tmp = NamedTemporaryFile()
    ase.io.write(tmp.name, atoms, format='png') # does not accept StringIO
    raw = open(tmp.name).read()
    tmp.close()
    return b64encode(raw)

In [ ]:
############################   END OF PREPROCESSING   ###############################

In [ ]:
def search():

    results.value = "preprocessing..."
    preprocess_newbies()
    try:
        import apps.scanning_probe.common
        apps.scanning_probe.common.preprocess_spm_calcs(workchain_list = ['OrbitalWorkChain'])
    except:
        print("Warning: scanning_probe app not found, skipping spm preprocess.")
    
    results.value = "searching..."
    
    # html table header
    html  = '<style>#aiida_results td,th {padding: 2px}</style>' 
    html += '<table border=1 id="aiida_results" style="margin:10px;"><tr>'
    html += '<th>PK</th>'
    html += '<th>Creation Time</th>'
    html += '<th>Formula</th>'
    html += '<th>CalcName</th>'
    html += '<th>Energy (Hrt)</th>'
   # html += '<th>Cell</th>'
    html += '<th>Structure</th>'
    html += '<th>Extra calculations</th>'
    html += '</tr>'

    # query AiiDA database
    filters = {}
    filters['attributes._process_label'] = 'MoleculeOptWorkChain'
    filters['extras.preprocess_version'] = PREPROCESS_VERSION
    filters['extras.preprocess_successful'] = True
    filters['extras.obsolete'] = False
    
    pk_list = inp_pks.value.strip().split()
    if pk_list:
        filters['id'] = {'in': pk_list}
        
    formula_list = inp_formula.value.strip().split()
    if inp_formula.value:
        filters['extras.formula'] = {'in': formula_list}
        
    if len(text_description.value) > 1:
        filters['description'] = {'like': '%{}%'.format(text_description.value)}

    qb = QueryBuilder()        
    qb.append(WorkCalculation, filters=filters)
    qb.order_by({WorkCalculation:{'ctime':'desc'}})

    for i, node_tuple in enumerate(qb.iterall()):
        node = node_tuple[0]
        thumbnail = node.get_extra('thumbnail')
        description = node.get_extra('structure_description')
        opt_structure_uuid = node.get_extra('opt_structure_uuid')
        
        ## Find all extra calculations done on the optimized geometry
        extra_calc_links = ""
        opt_structure = load_node(opt_structure_uuid)
        st_extras = opt_structure.get_extras()
        
        ### --------------------------------------------------
        ### add links to SPM calcs
        try:
            import apps.scanning_probe.common
            extra_calc_links += apps.scanning_probe.common.create_viewer_link_html(st_extras, "../../")
        except Exception as e:
            pass
        ### --------------------------------------------------
        
        extra_calc_area = "<div id='wrapper' style='overflow-y:auto; height:100px; line-height:1.5;'> %s </div>" % extra_calc_links
        
        # append table row
        html += '<tr>'
        html += '<td>%d</td>' % node.pk
        html += '<td>%s</td>' % node.ctime.strftime("%Y-%m-%d %H:%M")
        html += '<td>%s</td>' % node.get_extra('formula')
        html += '<td>%s</td>' % node.description
        html += '<td>%f</td>' % node.get_extra('energy')
       # html += '<td>%s %f</td>' % (node.get_extra('cell'),node.get_extra('volume'))
        html += '<td style="text-align: center;"><a target="_blank" href="./export_structure.ipynb?uuid=%s">'%opt_structure_uuid
        html += '<img width="auto" style="max-height: 400px; max-width: 300px;" src="data:image/png;base64,%s" title="PK%d: %s"></a></td>' % (thumbnail, opt_structure.pk, description)
        html += '<td>%s</td>' % extra_calc_area
        html += '</td>'
        html += '</tr>'

    html += '</table>'
    html += 'Found %d matching entries.<br>'%qb.count()

    results.value = html

In [ ]:
# search UI
style = {"description_width":"100px"}
layout = ipw.Layout(width="592px")
inp_pks = ipw.Text(description='PKs', placeholder='e.g. 4062 4753 (space separated)', layout=layout, style=style)
inp_formula = ipw.Text(description='Formulas:', placeholder='e.g. C44H16 C36H4', layout=layout, style=style)
text_description = ipw.Text(description='Calculation Name: ',
                            placeholder='e.g. a great name.',
                            layout=layout, style=style)
search_crit = [inp_pks, inp_formula, text_description]

In [ ]:
def on_click(b):
    with info_out:
        clear_output()
        search()

button = ipw.Button(description="Search")
button.on_click(on_click)
results = ipw.HTML()
info_out = ipw.Output()
app = ipw.VBox(children=search_crit + [button, results, info_out])
display(app)